In [14]:
!pip install streamlit -q

In [15]:
!pip install pmdarima

In [16]:
!pip install neattext

In [17]:
%%writefile e.py
import streamlit as st

import base64

@st.cache(allow_output_mutation=True)
def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()

def set_png_as_page_bg(png_file):
    bin_str = get_base64_of_bin_file(png_file)
    page_bg_img = '''
    <style>
    .stApp {
      background-image: url("data:image/png;base64,%s");
      background-size: cover;
    }
    </style>
    ''' % bin_str
    
    st.markdown(page_bg_img, unsafe_allow_html=True)
    return

set_png_as_page_bg('background.png')

Overwriting e.py


In [18]:
%%writefile home.py
import streamlit as st
from PIL import Image

def app():
    st.title('A Covid-19 Study and Forecasting')  
    st.write('Covid-19 has proven to be a particularly challenging epidemic to contain. Covid-19 has been around for more than two years, despite the swift efforts of medical specialists. Our project  helps this cause by presenting ways to forecast the estimated number of cases in a certain state or country on a future date. This will contribute in the preparation and fight against the spread of the virus. The project will also provide statistics on the many patterns that have been observed in India as a result of the Covid-19 problem, as well as how it has affected the people.')
    image = Image.open('covid pic.png')
    st.image(image)
    st.write('By knowing more about the virus, we will be able to comprehend it’s different characteristics and form tactics to better handle the situation. Moreover by analysing the social attitude towards the virus, we will also be able to handle the decisions taken that affect the common public. Also, this project can help deal in anticipating people’s reaction in case of future epidemics/pandemics. Forecasting of the virus proves its usefulness as well in terms of predicting when the virus calms down and when it can spike and people and governments can take precautions to address that.')


Overwriting home.py


In [19]:
%%writefile graphs.py
import streamlit as st
import pandas as pd
import os
import gc
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, Markdown

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot

def app():
    st.title("Visualizations")

    state_test = pd.read_csv('StatewiseTestingDetails.csv')
    state_vacc = pd.read_csv('covid_vaccine_statewise.csv')
    covid = pd.read_csv('covid_19_india.csv')

    covid.columns = ['Sno', 'Date', 'Time', 'State', 'CnfrmIndian', 'CnfrmForeign', 'Cured', 'Death', 'Confirmed']
    state_vacc.columns = [x.strip().replace(' ','_') for x in state_vacc.columns]

    state_vacc['Updated_On'] = pd.to_datetime(state_vacc['Updated_On'], format='%d/%m/%Y')
    state_vacc['Month'] = state_vacc['Updated_On'].dt.month
    month_val = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug']
    month = {i+1:x for i,x in enumerate(month_val)}
    state_vacc['Month'] = state_vacc['Month'].map(month)
    #state_vacc['Updated_On'] = state_vacc['Updated_On'].astype(str)
    #state_vacc['Month'] = state_vacc['Updated_On'].apply(lambda x:x.split('-')[0]) + state_vacc['Month']

    covid['Date'] = pd.to_datetime(covid['Date'], format='%Y/%m/%d')
    covid['Month'] = covid['Date'].dt.month
    month_val_new = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
    month = {i+1:x for i,x in enumerate(month_val_new)}
    covid['Month'] = covid['Month'].map(month)
    #covid['Date'] = covid['Date'].astype(str)
    #covid['Month'] = covid['Date'].apply(lambda x:x.split('-')[0]) + covid['Month']

    STATE = 'Kerala'

    state_test['Negative'] = state_test['Negative'].astype(str)
    state_test['Negative'] = state_test['Negative'].apply(lambda x:x.replace('nan','-1'))
    state_test['Negative'] = state_test['Negative'].apply(lambda x:x.replace(' ','-1'))
    state_test['Negative'] = state_test['Negative'].astype(int)

    covid['CnfrmIndian'] = covid['CnfrmIndian'].astype(str)
    covid['CnfrmIndian'] = covid['CnfrmIndian'].apply(lambda x:x.replace('-','0'))
    covid['CnfrmIndian'] = covid['CnfrmIndian'].astype(int)

    covid['CnfrmForeign'] = covid['CnfrmForeign'].astype(str)
    covid['CnfrmForeign'] = covid['CnfrmForeign'].apply(lambda x:x.replace('-','0'))
    covid['CnfrmForeign'] = covid['CnfrmForeign'].astype(int)

    covid['State']=covid['State'].replace(['Bihar****','Madhya Pradesh***','Maharashtra***','Karanataka'],['Bihar','Madhya Pradesh','Maharashtra','Karnataka'])

    state_test['TotalSamples']=state_test['TotalSamples'].fillna(0)
    state_test['TotalSamples']=state_test['TotalSamples'].astype('int')
    state_test['Positive']=state_test['Positive'].fillna(0)
    state_test['Positive']=state_test['Positive'].astype('int')
    state_test['Negative']=state_test['Negative'].fillna(0)
    #state_testing['Negative']=state_testing['Negative'].astype('int')
    state_test['Date'] = pd.to_datetime(state_test['Date'], format='%Y/%m/%d')

    state_vacc['FDA_Each_Day']=state_vacc.groupby('State',sort=True)['First_Dose_Administered'].diff().reset_index(drop=True)
    state_vacc['SDA_Each_Day']=state_vacc.groupby('State',sort=True)['Second_Dose_Administered'].diff().reset_index(drop=True)
    covid['Confirmed_Each_Day']=covid.groupby('State',sort=True)['Confirmed'].diff().reset_index(drop=True)
    covid['Death_Each_Day']=covid.groupby('State',sort=True)['Death'].diff().reset_index(drop=True)
    state_vacc[state_vacc['State']=='Kerala'][['Updated_On','State','First_Dose_Administered','FDA_Each_Day']].head(10)

    df=covid
    df['Active_cases']=df['Confirmed']-(df['Cured']+df['Death'])

    st.subheader("Top 10 highly impacted states as on 11th August (Latest date in the dataset)")
    top_10=df.groupby(by='State').last()[['Active_cases','Date']].sort_values(by=['Active_cases'],ascending=False).reset_index()
    fig=plt.figure(figsize=(15,8))
    ax=sns.barplot(data=top_10.iloc[:10],y='Active_cases',x='State', linewidth=2, palette='Dark2')
    ax.set_xticklabels(labels=ax.get_xticklabels(),rotation=90)
    st.pyplot(fig)

    st.subheader("Active Cases by States")
    state1 = st.selectbox("Pick state-1", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    state2 = st.selectbox("Pick state-2", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    state3 = st.selectbox("Pick state-3", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    state4 = st.selectbox("Pick state-4", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    fig=plt.figure(figsize=(15,8))
    ax=sns.lineplot(data=df[df['State'].isin([state1, state2, state3, state4])],x='Date',y='Active_cases',hue='State')
    st.pyplot(fig)

    st.subheader("Bar Plot of Sample States")
    state21 = st.selectbox("Pick state 1", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    state22 = st.selectbox("Pick state 2", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    state23 = st.selectbox("Pick state 3", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    state24 = st.selectbox("Pick state 4", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    fig=plt.figure(figsize=(15,8))
    sns.boxplot(data=df[df['State'].isin([state21, state22, state23, state24])],x=(df.Cured | df.Death),y='State')
    plt.xlabel('Cured/Deaths')
    st.pyplot(fig)

    st.subheader("Pivot table of confirmed and cured cases, deaths, the recovery rate and mortality rate")
    statewise = pd.pivot_table(covid, values=['Confirmed','Death','Cured'], index='State', aggfunc='max')
    statewise['Recovery Rate'] = statewise['Cured']*100 / statewise['Confirmed']
    statewise['Mortality Rate'] = statewise['Death']*100 /statewise['Confirmed']
    statewise = statewise.sort_values(by='Confirmed', ascending= False)
    statewise.style.background_gradient(cmap='YlOrRd')
    st.table(statewise)

    st.subheader("Statewise Testing Stats")
    testing=state_test.groupby('State')['TotalSamples'].max().sort_values(ascending=False).reset_index()
    fig = px.bar(testing, 
             x="TotalSamples",
             y="State", 
             orientation='h',
             height=800,
             width=800,             
             color='State')
    st.plotly_chart(fig)

    st.subheader("Comparing Positve and Negative Test Samples")
    fig=plt.figure(figsize=(20,10))
    plt.plot(state_test['TotalSamples'], label='Total Samples Checked')
    plt.plot(state_test['Negative'], label = 'Negative Samples')
    plt.plot(state_test['Positive'], label = 'Positive Samples')
    plt.legend()
    st.pyplot(fig)

    st.subheader("Trend of Corona virus in India")
    new_covid = covid.groupby('Date')['Confirmed','Cured','Death'].sum().reset_index()
    fig = plt.figure(figsize=(15,8))
    plt.plot(new_covid['Date'],new_covid['Confirmed'])
    plt.plot(new_covid['Date'],new_covid['Cured'])
    plt.plot(new_covid['Date'],new_covid['Death'])
    plt.xlabel('Date')
    plt.ylabel('Number of Cases')
    plt.legend(['Confirmed','Cured','Death'])        
    st.pyplot(fig)

    st.subheader("Trend of Corona virus Testing in India")
    new_state_test = state_test.groupby('Date')['TotalSamples','Positive'].sum().reset_index()
    fig = plt.figure(figsize=(15,8))
    plt.plot(new_state_test['Date'],new_state_test['TotalSamples'])
    plt.plot(new_state_test['Date'],new_state_test['Positive'])
    plt.xlabel('Date')
    plt.ylabel('Number of Samples')
    plt.legend(['Samples','Positive'])
    st.pyplot(fig)

    st.subheader("Confirmed cases and vaccination in different states")
    state3 = st.selectbox("Pick", ("Andhra Pradesh", "Arunachal Pradesh", "Bihar", "Chhattisgarh", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telengana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal"))
    states = [state3]
    fig=plt.figure(figsize=(20,10))
    for i, s in enumerate(states):
        sv = state_vacc[state_vacc['State'] == s]
        cv = covid[(covid['State'] == s) & (covid['Date'].dt.year>2020)]    
        total_dose_1 = sv.groupby('Month', sort=False)['FDA_Each_Day'].median()
        total_dose_2 = sv.groupby('Month', sort=False)['SDA_Each_Day'].median()
        confirmed = cv.groupby('Month', sort=False)['Confirmed_Each_Day'].median()
    
        _ = plt.subplot(2,3, i+1)
        _ = plt.title(f'Confirmed Case and Vaccination in {s}')
        _ = plt.bar(month_val, confirmed, label = 'Confirmed Case')
        _ = plt.bar(month_val, total_dose_1, label='First Dose', width=0.6*0.8)
        _ = plt.bar(month_val, total_dose_2, label='Second Dose', width=0.4*0.8)
        _ = plt.legend()
    _ = plt.tight_layout()
    st.pyplot(fig)

Overwriting graphs.py


In [20]:
%%writefile model.py
import streamlit as st
import pandas as pd
import os
import gc
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, Markdown

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot

from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
# from pmdarima.arima import auto_arima

# !pip3 install pyramid-arima
# !pip install pmdarima
# from statsmodels.tsa.arima_model import ARIMA
# import pmdarima as pm

import statsmodels.api as sm

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def app():
    st.title("Models")

    state_test = pd.read_csv('StatewiseTestingDetails.csv')
    state_vacc = pd.read_csv('covid_vaccine_statewise.csv')
    covid = pd.read_csv('covid_19_india.csv')

    covid.columns = ['Sno', 'Date', 'Time', 'State', 'CnfrmIndian', 'CnfrmForeign', 'Cured', 'Death', 'Confirmed']
    state_vacc.columns = [x.strip().replace(' ','_') for x in state_vacc.columns]

    state_vacc['Updated_On'] = pd.to_datetime(state_vacc['Updated_On'], format='%d/%m/%Y')
    state_vacc['Month'] = state_vacc['Updated_On'].dt.month
    month_val = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug']
    month = {i+1:x for i,x in enumerate(month_val)}
    state_vacc['Month'] = state_vacc['Month'].map(month)
    #state_vacc['Updated_On'] = state_vacc['Updated_On'].astype(str)
    #state_vacc['Month'] = state_vacc['Updated_On'].apply(lambda x:x.split('-')[0]) + state_vacc['Month']

    covid['Date'] = pd.to_datetime(covid['Date'], format='%Y/%m/%d')
    covid['Month'] = covid['Date'].dt.month
    month_val_new = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
    month = {i+1:x for i,x in enumerate(month_val_new)}
    covid['Month'] = covid['Month'].map(month)
    #covid['Date'] = covid['Date'].astype(str)
    #covid['Month'] = covid['Date'].apply(lambda x:x.split('-')[0]) + covid['Month']

    STATE = 'Kerala'

    state_test['Negative'] = state_test['Negative'].astype(str)
    state_test['Negative'] = state_test['Negative'].apply(lambda x:x.replace('nan','-1'))
    state_test['Negative'] = state_test['Negative'].apply(lambda x:x.replace(' ','-1'))
    state_test['Negative'] = state_test['Negative'].astype(int)

    covid['CnfrmIndian'] = covid['CnfrmIndian'].astype(str)
    covid['CnfrmIndian'] = covid['CnfrmIndian'].apply(lambda x:x.replace('-','0'))
    covid['CnfrmIndian'] = covid['CnfrmIndian'].astype(int)

    covid['CnfrmForeign'] = covid['CnfrmForeign'].astype(str)
    covid['CnfrmForeign'] = covid['CnfrmForeign'].apply(lambda x:x.replace('-','0'))
    covid['CnfrmForeign'] = covid['CnfrmForeign'].astype(int)

    covid['State']=covid['State'].replace(['Bihar****','Madhya Pradesh***','Maharashtra***','Karanataka'],['Bihar','Madhya Pradesh','Maharashtra','Karnataka'])

    state_test['TotalSamples']=state_test['TotalSamples'].fillna(0)
    state_test['TotalSamples']=state_test['TotalSamples'].astype('int')
    state_test['Positive']=state_test['Positive'].fillna(0)
    state_test['Positive']=state_test['Positive'].astype('int')
    state_test['Negative']=state_test['Negative'].fillna(0)
    #state_testing['Negative']=state_testing['Negative'].astype('int')
    state_test['Date'] = pd.to_datetime(state_test['Date'], format='%Y/%m/%d')

    state_vacc['FDA_Each_Day']=state_vacc.groupby('State',sort=True)['First_Dose_Administered'].diff().reset_index(drop=True)
    state_vacc['SDA_Each_Day']=state_vacc.groupby('State',sort=True)['Second_Dose_Administered'].diff().reset_index(drop=True)
    covid['Confirmed_Each_Day']=covid.groupby('State',sort=True)['Confirmed'].diff().reset_index(drop=True)
    covid['Death_Each_Day']=covid.groupby('State',sort=True)['Death'].diff().reset_index(drop=True)
    state_vacc[state_vacc['State']=='Kerala'][['Updated_On','State','First_Dose_Administered','FDA_Each_Day']].head(10)

    df=covid
    df['Active_cases']=df['Confirmed']-(df['Cured']+df['Death'])

    st.subheader("FB Prophet")
    st.write('FB Prophet is used to determine time-series data on the basis of an additive model for which trends that are nonlinear are fit with regard to annually, weekly, daily-seasonality and holiday effects. It delivers its best work with time-series that show reliable regular impacts and a few seasons of recorded information. Prophet can deal with data that’s missing and shifts in trend, and can handle exceptions well.')
    Trend = covid.groupby(['Date'])['Confirmed', 'Death','Cured',].sum().reset_index()
    Confirmed = Trend.loc[:, ['Date', 'Confirmed']] 
    Confirmed.columns = ['ds', 'y']

    from sklearn.preprocessing import MinMaxScaler
    values = Confirmed['y'].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    model = scaler.fit(values)
    Confirmed['y'] = scaler.transform(values)

    test_size = 30
    train = Confirmed.iloc[:-test_size]
    test = Confirmed.iloc[-test_size:]
    test.reset_index(inplace = True)

    prop=Prophet()
    prop.fit(train)
    future=prop.make_future_dataframe(periods=60)
    new_prop_forecast = prop.predict(future.iloc[530:560])
    new_prop_forecast['inverse_scaled_yhat'] = scaler.inverse_transform(new_prop_forecast['yhat'].values.reshape(-1, 1))

    plt.figure(figsize= (15,10))
    plt.xlabel("Dates",fontsize = 20)
    plt.ylabel('Total Cases',fontsize = 20)
    plt.title("Predicted Values for the next 30 Days" , fontsize = 20)
    plt.plot_date(x=test['ds'],y=scaler.inverse_transform(test['y'].values.reshape(-1, 1)),color='blue',label='Actual',linestyle='solid')
    plt.plot_date(x = new_prop_forecast['ds'], y = new_prop_forecast['inverse_scaled_yhat'], color = 'yellow', label = 'Predicted',linestyle = 'solid')
    plt.legend()

    st.pyplot(plt)

    st.subheader("Auto Arima")
    st.write('ARIMA model is a widely used and prevalent statistical method for time-series forecasting. The name of the model is an acronym for Auto Regressive Integrated Moving Average. It belongs to a class of models which captures diverse standard temporal structures in data which is time series.')

    start_date = Confirmed['ds'].max()
    prediction_dates = []
    for i in range(30):
        date = start_date + datetime.timedelta(days=1)
        prediction_dates.append(date)
        start_date = date

    auto_arima_model = pm.auto_arima(train['y'], start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=False,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True)
    
    arima = ARIMA(train['y'], order=(1, 1, 1)).fit()
    forecast = arima.forecast(steps=30)
    arima_future= arima.predict(len(train)+1,end=len(train)+(30), dynamic= False)

    plt.figure(figsize= (15,10))
    plt.xlabel("Dates",fontsize = 20)
    plt.ylabel('Total Cases',fontsize = 20)
    plt.title("Predicted Values for the next 30 Days" , fontsize = 20)

    plt.plot_date(x=test['ds'],y=scaler.inverse_transform(test['y'].values.reshape(-1, 1)),color='blue',label='Actual',linestyle='solid')
    plt.plot_date(x=test['ds'],y=scaler.inverse_transform(arima_future.values.reshape(-1, 1)),color='yellow',label='Predicted',linestyle='solid')
    plt.legend()
    st.pyplot(plt)    

    st.subheader("Sarima")
    st.write('Since ARIMA does not support seasonal data, we use SARIMA(Seasonal Autoregressive Integrated Moving Average). SARIMA is an extension of ARIMA that expressly underpins univariate time series data with a seasonal component.')

    SARIMAX__model = sm.tsa.statespace.SARIMAX(train['y'].values,
                                            order=(1,1,1),
                                            seasonal_order=(1,0,1,7),
                                         enforce_stationarity=False,
                                         enforce_invertibility=False,)
    SARIMAX__model = SARIMAX__model.fit(maxiter=1000)

    days=30
    prediction = SARIMAX__model.get_forecast(steps=days)
    pred_date = prediction.summary_frame(alpha=0.05).set_index(pd.date_range(start='2021-08-11',periods=days,freq='D'))
    sarima_train=SARIMAX__model.predict(start=len(train)+1 ,end=len(train)+30, dynamic= False)

    plt.figure(figsize= (15,10))
    plt.xlabel("Dates",fontsize = 20)
    plt.ylabel('Total Cases',fontsize = 20)
    plt.title("Predicted Values for the next 30 Days" , fontsize = 20)

    plt.plot_date(x=test['ds'],y=scaler.inverse_transform(test['y'].values.reshape(-1, 1)),color='blue',label='Actual',linestyle='solid')
    plt.plot_date(x=test['ds'],y=scaler.inverse_transform(sarima_train.reshape(-1, 1)),color='yellow',label='Predicted',linestyle='solid')
    plt.legend()
    st.pyplot(plt)    

    st.subheader("Results")
    d = {"mae": [0.19325423865595204,0.0015789614203266253,0.001092248884050632], 
         "mape": [0.19622294372659527,0.0015940655126828426,0.001115850924221506], 
         "me": [0.19325423865595204,-0.0015754530399724148,0.0007217380566784856], 
         "minmax": [0.16286404180025793,0.0010894829966355823,0.001114495086385059], 
         "mpe": [0.19622294372659527,-0.001590433508638211,0.0007444060020857142],
         "rmse": [0.19865221424641694,0.00197054372299962,0.0012089738445670137]}

    df = pd.DataFrame(d, index=["FB Prophet","Auto Arima","Sarima"])
    st.table(df)

Overwriting model.py


In [21]:
%%writefile twitter.py
import streamlit as st
import os
import gc
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date
import re
from textblob import TextBlob
from tweepy import OAuthHandler
import warnings
import neattext.functions as nfx
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from textblob import TextBlob

def app():
    df = pd.read_csv('/content/drive/MyDrive/FYP Datasets/vaccination_all_tweets.csv')

    df.dropna(inplace=True)
    df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
    #len(df[df['created_at'].str.contains('India',case=False)])
    df_india = df[df['user_location'].str.contains('india|andhra pradesh|arunachal pradesh|bihar|chhattisgarh|goa|gujarat|haryana|himachal pradesh|jharkhand|karnataka|kerala|assam|madhya pradesh|maharashtra|manipur|meghalaya|mizoram|nagaland|odisha|punjab|rajasthan|sikkim|tamil nadu|telangana|tripura|uttarakhand|uttar pradesh|west bengal|andaman and nicobar island|chandigarh|dadra and nagar haveli and daman & diu|delhi|jammu and kashmir|ladakh|lakshawadweep|puducheri',case=False)]
    #df_india = df
    #len(df[df['user_location'].str.contains('india|delhi')])
    df_india = df_india[(df_india['date'].dt.year > 2020) & (df_india['date'].dt.year < 2022)]
    df_india.reset_index(inplace = True)
    df_india.drop(labels = 'index', axis = 1)
    re.sub("RT @[\w]*:", "" , df_india.text[0])

    def clean_data(tweet):
        tweet = re.sub("RT @[\w]*:", "", tweet)
        tweet = re.sub("@[\w]*", "", tweet)
        tweet = re.sub("https://[A-Za-z0-9./]", "", tweet)
        tweet = re.sub("\n", "", tweet)
        tweet = re.sub("&amp", "", tweet)
        tweet = re.sub("#", "", tweet)
        tweet = re.sub(r"[^\w]", ' ', tweet )
        return tweet

    df_india['text']=df_india['text'].apply(lambda x:clean_data(x))
    df_india['extracted_hashtags'] = df_india['text'].apply(nfx.extract_hashtags)
    df_india['clean_tweet'] = df_india['text'].apply(nfx.remove_hashtags)
    df_india['clean_tweet'] = df_india['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x))
    df_india['clean_tweet'] = df_india['clean_tweet'].apply(nfx.remove_multiple_spaces)
    df_india['clean_tweet'] = df_india['clean_tweet'].apply(nfx.remove_urls)
    df_india['clean_tweet'] = df_india['clean_tweet'].apply(nfx.remove_puncts)

    def get_sentiment(text):
        blob = TextBlob(text)
        sentiment_polarity = blob.sentiment.polarity
        sentiment_subjectivity = blob.sentiment.subjectivity
        if sentiment_polarity > 0:
            sentiment_label = 'Positive'
        elif sentiment_polarity < 0:
            sentiment_label = 'Negative'
        else:
            sentiment_label = 'Neutral'
        result = {'polarity':sentiment_polarity,
                  'subjectivity':sentiment_subjectivity,
                  'sentiment':sentiment_label}
        return result

    ex1 = df_india['clean_tweet'].iloc[0]
    get_sentiment(ex1)
    df_india['sentiment_results'] = df_india['clean_tweet'].apply(get_sentiment)
    df_india = df_india.join(pd.json_normalize(df_india['sentiment_results']))

    st.subheader('Sentiment Towards Vaccine (2021)')
    fig=plt.figure(figsize=(18,10))
    sns.countplot(df_india['sentiment'])
    plt.ylabel("Count")
    plt.xlabel("Sentiment")
    st.pyplot(fig)

    df_india['month'] = df_india['date'].dt.month
    df_india['sentiment'] = df_india['sentiment'].astype('category')
    df_india_byMonth = df_india.groupby(['month','sentiment']).count()
    df_india_byMonth.reset_index(inplace = True)

    sns.barplot(x = 'sentiment',y = 'index', data = df_india_byMonth[df_india_byMonth['month']==9])

    st.subheader('Sentiment Towards Vaccine (Monthwise)')
    x = st.selectbox("Pick", ("January", "February", "March", "April", "May", "June", "July", "August"))
    d = {"January":1, "February":2, "March":3, "April":4, "May":5, "June":6, "July":7, "August":8}
    fig=plt.figure(figsize=(18,10))
    i=d[x]
    plt.subplot(3,3, i)
    plt.title(x)
    sns.barplot(x = 'sentiment',y = 'index', data = df_india_byMonth[df_india_byMonth['month']==i])
    plt.ylabel("Count")
    plt.xlabel("Sentiment")
    st.pyplot(fig)

Overwriting twitter.py


In [22]:
%%writefile multiapp.py
import streamlit as st
import tornado.ioloop
from tornado.iostream import IOStream

class MultiApp:
    def __init__(self):
        self.apps = []

    def add_app(self, title, func):
        self.apps.append({
            "title": title,
            "function": func
        })

    def run(self):
        # app = st.sidebar.radio(
        app = st.sidebar.selectbox(
            'Navigation',
            self.apps,
            format_func=lambda app: app['title'])

        app['function']()

app = MultiApp()

import home
import graphs
import model
import twitter
# Add all your application here
app.add_app("Home", home.app)
app.add_app("Data Analysis and Visualization", graphs.app)
app.add_app("Forcasting", model.app)
app.add_app("Vaccine Sentiment Analysis", twitter.app)

# The main app
app.run()

Overwriting multiapp.py


In [13]:
!streamlit run multiapp.py & npx localtunnel --port 8501

2022-04-26 05:25:11.153 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.572s
your url is: https://smooth-bird-9.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.245.9.129:8501

2022-04-26 05:25:26.054 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/scriptrunner/script_runner.py", line 443, in _run_script
    exec(code, module.__dict__)
  File "/content/multiapp.py", line 37, in <module>
    app.run()
  File "/content/multiapp.py", line 22, in run
    app['function']()
  File "/content/home.py", line 7, in app
    image = Image.open('covid pic.jpg')
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2843, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'covid pic.jpg'

  Stopping...
^C


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
